Choice in Motors, ESCs, and Battery

High power motors were chosen for their size while having good power to price ratio for a few reasons:
* Need a high torque fast control system response.
* A large range of power allows for precise system control at higher speeds.
* Cost; we're students on a budget.

The ESCs were chosen to match the motors maximum power of the motors.
The battery discharge rate must also match the ESCs maximum discharge rate, however the battery capacity determines the time of flight and acceleration of the drone.

https://www.aliexpress.com/item/1005004476613305.html?spm=a2g0o.order_list.order_list_main.5.155a1802zZ51a7

Selecting optimal battery size

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# to be done
#
rho_frame = 1500 # density of wood
rho_blade = 1240 # density of PLA

blade_length = 5 * 25.4 # length of blade in mm
blade_width = 0.5 * 25.4 # width of blade in mm
blade_area_m2 = blade_length * blade_width / 1000**2 # area of blade in m2
blade_thickness = 3e-3 # mm
blade_mass = blade_area_m2 * blade_thickness * rho_blade

motor_mass = 29e-3 # kg
ESC_mass = 9e-3 # kg

frame_length = 1.2 * blade_length
frame_area = 0.5 * frame_length**2 / 1000**2 # area of frame in m2
frame_thickness = 5e-3
frame_mass = frame_area * frame_thickness * rho_frame

drone_mass = frame_mass + 4 * (blade_mass + motor_mass + ESC_mass)
print(drone_mass)

In [ ]:
# https://www.flyingtech.co.uk/sites/default/files/product_files/RS2205-S%20Specs.jpg
# using HQ5045BN propeller

v_3s = 11.1 # motor currents and thrusts for 3 cell battery
motor_currents_3s = np.array([1,3.1,5.1,7.1,9.1,11.1,13.1,15.1,17.1,19.1,21.1,22.5,23])
motor_thrusts_3s = np.array([72,176,255,329,396,460,527,575,632,681,725,758,774]) * 1e-3 # kg thrust

v_4s = 14.8 # motor currents and thrusts for a 4 cell battery
motor_currents_4s = np.array([1,3.1,5.1,7.1,9.1,11.1,13.1,15.2,17.1,19.1,21.2,23.2,25.2,27.3,29.3,31.3,33.3,35.1,36.4])
motor_thrusts_4s = np.array([90,204,306,388,457,523,591,664,722,774,836,887,942,995,1045,1098,1150,1208,1236]) * 1e-3 # kg thrust

# batteries are 100 to 158 Wh/Kg typically in the range of 50 to 500 g
edensity_lower = 100 # Wh/Kg
edensity_upper = 158 # Wh/Kg
battery_masses = np.linspace(50, 1000, 100) / 1000 # kg
battery_capacacties_lower_3s = battery_masses * edensity_lower * 1000 / v_3s # mAh
battery_capacacties_middle_3s = battery_masses * (edensity_upper + edensity_lower) * 500 / v_3s # mAh
battery_capacacties_upper_3s = battery_masses * edensity_upper * 1000 / v_3s # mAh

# polyfit and find current at each hover thrust for each battery mass
fit_3s = np.poly1d(np.polyfit(motor_thrusts_3s, motor_currents_3s, 5))
hover_currents_3s = fit_3s((battery_masses + drone_mass) / 4 ) # divide by 4 for 4 motors

power_3s = v_3s**2 * hover_currents_3s * 4 # 4 motors
TOF_upper_3s = battery_capacacties_lower_3s * v_3s * (60 / 1000) / power_3s
TOF_middle_3s = battery_capacacties_middle_3s * v_3s * (60 / 1000) / power_3s
TOF_lower_3s = battery_capacacties_upper_3s * v_3s * (60 / 1000) / power_3s

battery_capacacties_lower_4s = battery_masses * edensity_lower * 1000 / v_4s # mAh
battery_capacacties_middle_4s = battery_masses * (edensity_upper + edensity_lower) * 500 / v_4s # mAh
battery_capacacties_upper_4s = battery_masses * edensity_upper * 1000 / v_4s # mAh

# polyfit and find current at each hover thrust for each battery mass
fit = np.poly1d(np.polyfit(motor_thrusts_4s, motor_currents_4s, 5))
hover_currents_4s = fit((battery_masses + drone_mass) / 4 ) # divide by 4 for 4 motors

power_4s = v_4s**2 * hover_currents_4s * 4 # 4 motors
TOF_upper_4s = battery_capacacties_lower_4s * v_4s * (60 / 1000) / power_4s
TOF_middle_4s = battery_capacacties_middle_4s * v_4s * (60 / 1000) / power_4s
TOF_lower_4s = battery_capacacties_upper_4s * v_4s * (60 / 1000) / power_4s

plt.fill_between(battery_capacacties_lower_3s, TOF_lower_3s, TOF_upper_3s, alpha=0.4, label='3 cell (11.1V)')
plt.plot(battery_capacacties_middle_3s, TOF_middle_3s, 'k', c='blue')

plt.fill_between(battery_capacacties_lower_4s, TOF_lower_4s, TOF_upper_4s, alpha=0.4, label='4 cell (14.8V)')
plt.plot(battery_capacacties_middle_4s, TOF_middle_4s, 'k', c='r')

plt.xlabel('Battery capacity (mAh)')
plt.ylabel('Time of flight (min)')
plt.xlim(0, 6000)
plt.legend()


In [ ]:
# graph of max acceleration vs. battery capacity

fit_3s = np.poly1d(np.polyfit(motor_currents_3s, motor_thrusts_3s, 5))
fit_4s = np.poly1d(np.polyfit(motor_currents_4s, motor_thrusts_4s, 5))
max_thrust_3s = fit_3s(motor_currents_3s[-1]) # 3 cell battery
max_thrust_4s = fit_4s(motor_currents_4s[-1]) # 4 cell battery

g = 9.81
max_accelerations_3s = ((max_thrust_3s * 4) / (battery_masses + drone_mass) - 1 ) * g # 4 motors
max_accelerations_4s = ((max_thrust_4s * 4) / (battery_masses + drone_mass) - 1 ) * g # 4 motors

plt.plot(battery_capacacties_middle_3s, max_accelerations_3s, label='3 cell (11.1V)')
plt.plot(battery_capacacties_middle_4s, max_accelerations_4s, label='4 cell (14.8V)')

plt.xlabel('Battery capacity (mAh)')
plt.ylabel('Max acceleration (m/s^2)')
plt.legend()
